In [5]:
n = 3
goodbasis_threshold = 0.9
badbasis_threshold = 1e-10
size = 100

def hadamard_ratio(L):
    H = abs(L.det())
    for v in L:
        H /= v.norm()
    return ((H ** (1 / L.dimensions()[0]))).n()

def babai(L, w):
    a = w * L.inverse()
    a = a.apply_map(lambda x : x.round())
    return (a, a * L)

In [6]:
def random_sl(n, ops, limit):
    """
    Generates a random n x n matrix with determinant 1. 
    """
    L = identity_matrix(ZZ, n)
    for _ in range(ops):
        i = randrange(n)
        j = randrange(n)
        if i == j: 
            continue
        L[i] += L[j] * randrange(-limit, limit)
    return L

In [7]:
goodbasis = random_matrix(ZZ, n, x=-size, y=size)

while hadamard_ratio(goodbasis) < goodbasis_threshold:
    goodbasis = random_matrix(ZZ, n, x=-size, y=size)

hadamard_ratio(goodbasis)

0.910176977865485

In [8]:
goodbasis

[ 68   0 -45]
[ 51  -2  55]
[ 61 -79 -25]

In [9]:
U = random_sl(n, 15, 100)
badbasis = U * goodbasis

while hadamard_ratio(badbasis) > badbasis_threshold:
    U = random_sl(n, 15, 100)
    badbasis = U * goodbasis

hadamard_ratio(badbasis)

9.50899190268498e-11

In [10]:
U

[   115467533    -93359954  12356420838]
[     1228378      -993191    131451285]
[  2886741636  -2334041954 308916225871]

In [11]:
badbasis

[   756832105708   -975970526294   -319241357405]
[     8051405348    -10382665133     -3396184640]
[ 18921152069725 -24399713759901  -7981181327865]

In [12]:
hadamard_ratio(badbasis)

9.50899190268498e-11

In [13]:
def sign(d):
    """
    Signs document d
    """
    return babai(goodbasis, d)[0] * U.inverse()

In [14]:
def sign_eve(d):
    return babai(badbasis, d)[0]

In [15]:
def verify(d, s):
    return (d - s * badbasis).norm().n()

In [16]:
d = random_vector(n, x=-10**6, y=10**6)
d

(188904, 973626, 632830)

In [17]:
s = sign(d)
s

(11919303804469, 16673781986, -476770442468)

In [18]:
verify(d, s)

30.4138126514911

In [19]:
se = sign_eve(d)

In [20]:
verify(d, se)

1.03583091602060e12

In [21]:
def e(m):
    return m * badbasis + random_vector(ZZ, 3, x=-5, y=5)

def d(c):
    return babai(goodbasis, c)[0] * U.inverse()

In [22]:
def d_eve(c):
    return babai(badbasis, c)[0]

In [23]:
m = random_vector(3, x=-10**6, y=10**6)
m

(-183558, 111065, -279367)

In [24]:
c = e(m)
c

(-5423973848587437519, 6994468881124740072, 2287900791857166844)

In [25]:
assert(d(c) == m)
d(c)

(-183558, 111065, -279367)

In [26]:
assert(d_eve(c) != m)
d_eve(c)

(-14593291, 90908, 297020)

In [64]:
def gauss(L):
    """
    2D Gaussian Lattice Reduction
    """
    while True:
        if L[1].norm() < L[0].norm():
            L[0], L[1] = L[1], L[0]
        mu = (L[1] * L[0]) / (L[0] * L[0])
        L[1] -= mu.round() * L[0]
        if -1/2 <= mu <= 1/2:
            return L

In [65]:
L = random_matrix(ZZ, 2, x=-100, y=100)
L

[-20  48]
[-80  91]

In [66]:
hadamard_ratio(L)

0.566219866764422

In [67]:
U = random_sl(2, 15, 100)
L_bad = U * L
L_bad

[ 24346460 -27761945]
[  -869840    991867]

In [68]:
hadamard_ratio(L_bad)

6.43947122586721e-6

In [69]:
L_better = gauss(L_bad)
L_better

[-40  -5]
[-20  48]

In [70]:
hadamard_ratio(L_better)

0.981659042857473